In [1]:
# MTF booking 2022
# 2021-10-31 делаю отчет для передачи в Телеграм по продажам туров за вчерашний день


In [2]:
#работаю в директории:
import os

os.chdir('C:\\Python\\MTF_telegram')
os.getcwd()


'c:\\Python\\MTF_telegram'

In [3]:
# загрузка библиотек
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
# import openpyxl
# import xlsxwriter

import pymysql.cursors

import psycopg2
import pandas.io.sql as psql

import requests

In [4]:
# подключение к view из MTF booking
conn = psycopg2.connect(dbname='XXX', user='XXX', password='XXX', host='XX.XX.XX.XX')
cursor = conn.cursor()

cursor.execute('SELECT * FROM payments_view')

records = cursor.fetchall()

conn_2 = psycopg2.connect(dbname='XXX', user='XXX', password='XXX', host='XX.XX.XX.XX')
df = psql.read_sql('SELECT * FROM payments_view', conn_2)

df.info()

#  2021 10 22 грузилось 9,5 сек.
#  2021 10 22 грузилось 7,7 сек.
#  2021 10 22 грузилось 8,1 сек.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133099 entries, 0 to 133098
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   doc_num        133099 non-null  int64  
 1   doc_date       133099 non-null  object 
 2   cruise_type    133099 non-null  object 
 3   summ           133099 non-null  float64
 4   date_start     133099 non-null  object 
 5   date_finish    133099 non-null  object 
 6   route          133099 non-null  object 
 7   ship_name      133099 non-null  object 
 8   payer          133088 non-null  object 
 9   operator_name  133099 non-null  object 
 10  manager        133096 non-null  object 
 11  orderid        133099 non-null  int64  
 12  tourid         133099 non-null  int64  
 13  ptype          133099 non-null  object 
 14  office         118379 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 15.2+ MB


In [5]:
#преобразовываю в dstetime
df["doc_date"] = pd.to_datetime(df["doc_date"])
df["date_start"] = pd.to_datetime(df["date_start"])
df["date_finish"] = pd.to_datetime(df["date_finish"])

#добавляю год и месяц в отдельные столбцы
df['Month'] = df["doc_date"].dt.month
df['Year'] = df["doc_date"].dt.year
df['Day'] = df["doc_date"].dt.day


df['Month_start'] = df["date_start"].dt.month
df['Year_start'] = df["date_start"].dt.year


In [6]:
#столбец с тыс.₽
df["ST"] = df["summ"]/1000

#округлить df["ST"]:
df["ST2"] = pd.to_numeric(df["ST"]).round(1)

In [7]:
#Табл. данные по круизам только с 2021 года и далее
df1 = df[df["date_start"] >= '2022-01-01']

In [8]:
# нахождение крайней даты
lrd = df1["doc_date"].sort_values(ascending=True).tail(1).astype('datetime64[ns]')

#lrd
print('lrd: ', lrd)
print('-------')

#привоэу в более менее удобный формат для выдачи в текст заголовка
lrd2 = lrd.astype('str').values

print('lrd2: ', lrd2)
print('-------')

lrd3 = lrd.astype('datetime64[ns]') #.dt.strftime('%Y/%m/%d')
print('lrd3: ', lrd3)
print('-------')

lrd:  74886   2022-01-11
Name: doc_date, dtype: datetime64[ns]
-------
lrd2:  ['2022-01-11']
-------
lrd3:  74886   2022-01-11
Name: doc_date, dtype: datetime64[ns]
-------


In [9]:
# сегодня
today = pd.to_datetime('today')

print('today: ', today)

today2 = datetime.today().strftime('%Y-%m-%d')

print('today2: ', today2)

today3 = datetime.today().strftime('%Y-%m-%d %H:%M')

print('today3: ', today3)


today:  2022-01-11 19:37:32.708559
today2:  2022-01-11
today3:  2022-01-11 19:37


In [10]:
type(today)
type(today2)
type(today3)

str

In [11]:
# вчера

yestoday = (datetime.today() - timedelta(days=1)) #.strftime('%Y-%m-%d')

print('yestoday: ', yestoday)

yestoday:  2022-01-10 19:37:38.904280


In [12]:
yestoday2 = yestoday.strftime('%Y-%m-%d')

yestoday2

'2022-01-10'

In [13]:
type(yestoday)

datetime.datetime

In [14]:
from pandas.tseries.offsets import MonthEnd, MonthBegin

dbm = pd.to_datetime(today2)-MonthEnd(1)

print('dbm: ', dbm)
print(type(dbm))

dbm:  2021-12-31 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [15]:
y = pd.to_datetime(today2)-timedelta(days=1)

y

Timestamp('2022-01-10 00:00:00')

In [16]:
#Табл с данными только по МТФ теплоходам стартующими с 2022 года и далее
df1MTF = df1[df1['operator_name'] == "ОАО 'Мостурфлот'"]

In [17]:
#надо добавать столбцы с полож и отриц суммами в ST2
#df2020MTF.where(df2020MTF['ST2']>0) # верно отрабатывает

#plus = df2020MTF.where(df2020MTF['ST2']>0)
#minus = df2020MTF.where(df2020MTF['ST2']<0)

df1MTF['ST2p'] = np.where(df1MTF['ST2']>0, df1MTF['ST2'], 0) # верно отрабатывает
df1MTF['ST2m'] = np.where(df1MTF['ST2']<0, df1MTF['ST2'], 0) # верно отрабатывает

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# df1MTF.info()

In [20]:
# делаю через  loc

df1MTF['ST2p'] = 0

df1MTF.loc[df1MTF['ST2']>0, 'ST2p'] =  df1MTF['ST2']

df1MTF['ST2m'] = 0

df1MTF.loc[df1MTF['ST2']<0, 'ST2m'] =  df1MTF['ST2']


C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Program Files\Python37\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [21]:
# df1MTF.info()

In [22]:
# df1MTF['doc_date']

In [23]:
# база только за вчерашний день
# df1.info()

# dfyt = df1MTF[df1MTF['doc_date'] > dbm]

dfy = df1MTF[df1MTF['doc_date'] == y]

# dftm.tail()

In [24]:
# dfy.info()
dfy.tail(20)

,doc_num,doc_date,cruise_type,summ,date_start,date_finish,route,ship_name,payer,operator_name,...,office,Month,Year,Day,Month_start,Year_start,ST,ST2,ST2p,ST2m
92823,145161,2022-01-10,rivercruises,55677.0,2022-08-01,2022-08-08,Москва (СРВ) - Углич - Кузино - Горицы - Мандр...,Николай КАРАМЗИН,Никитина Екатерина Сергеевна,ОАО 'Мостурфлот',...,None,1,2022,10,8,2022,55.677,55.7,55.7,0.0
93356,145158,2022-01-10,rivercruises,68985.0,2022-06-20,2022-06-25,Москва (СРВ) - Мышкин - Ярославль - Плёс - Кос...,Андрей РУБЛЕВ,Самошкин Алексей Павлович,ОАО 'Мостурфлот',...,None,1,2022,10,6,2022,68.985,69.0,69.0,0.0
96745,145199,2022-01-10,rivercruises,143352.0,2022-06-05,2022-06-10,Москва (СРВ) - Углич - Ярославль - Плёс - Кост...,Илья РЕПИН,"ООО ""Твой Круиз""",ОАО 'Мостурфлот',...,Таганка,1,2022,10,6,2022,143.352,143.4,143.4,0.0
97329,145181,2022-01-10,rivercruises,37800.0,2022-08-13,2022-08-19,Москва (СРВ) - Мышкин - Ярославль - Кострома -...,РОССИЯ,"ООО ""Инфофлот-С""",ОАО 'Мостурфлот',...,Таганка,1,2022,10,8,2022,37.800,37.8,37.8,0.0
103363,145193,2022-01-10,rivercruises,90396.0,2022-07-22,2022-07-30,Москва (СРВ) - Углич - Ярославль - Н.Новгород ...,Андрей РУБЛЕВ,ИП Воробьева Э.В.,ОАО 'Мостурфлот',...,Таганка,1,2022,10,7,2022,90.396,90.4,90.4,0.0
103944,145142,2022-01-10,rivercruises,64260.0,2022-07-22,2022-07-30,Москва (СРВ) - Углич - Ярославль - Н.Новгород ...,Андрей РУБЛЕВ,Рычкова Элла Николаевна,ОАО 'Мостурфлот',...,None,1,2022,10,7,2022,64.260,64.3,64.3,0.0
105430,145198,2022-01-10,rivercruises,124380.0,2022-06-04,2022-06-17,Москва (СРВ) - Углич - Горицы - Мандроги - о. ...,И.А. КРЫЛОВ,"ООО ""Твой Круиз""",ОАО 'Мостурфлот',...,Таганка,1,2022,10,6,2022,124.380,124.4,124.4,0.0
107728,145191,2022-01-10,rivercruises,83916.0,2022-05-10,2022-05-20,Москва (СРВ) - Углич - Кузино - Горицы - Мандр...,Михаил Булгаков,"ООО ""Твой Круиз""",ОАО 'Мостурфлот',...,Таганка,1,2022,10,5,2022,83.916,83.9,83.9,0.0
108592,145176,2022-01-10,rivercruises,26860.0,2022-06-10,2022-06-13,Москва (СРВ) - Дубна - Калязин - Мышкин - Угли...,Николай КАРАМЗИН,Винидиктов Александр Владимирович,ОАО 'Мостурфлот',...,None,1,2022,10,6,2022,26.860,26.9,26.9,0.0
113107,145165,2022-01-10,rivercruises,85050.0,2022-07-24,2022-07-31,Москва (СРВ) - Мышкин - Кострома - Плёс - Н.Но...,Княжна ВИКТОРИЯ,Добычина Марина Владимировна,ОАО 'Мостурфлот',...,None,1,2022,10,7,2022,85.050,85.0,85.0,0.0


In [25]:
# Платежи только за вчерашний день
pt1 = pd.pivot_table(dfy, index = ['ship_name'], values = ["ST2"], columns = ['Year', 'Month', 'Day'], aggfunc=np.sum, fill_value='', margins=False)

pt1

,ST2
Year,2022
Month,1
Day,10
ship_name,
Александр Грин,99.8
Андрей РУБЛЕВ,272.5
Василий СУРИКОВ,394.4
И.А. КРЫЛОВ,1350.3
Илья РЕПИН,89.8
Княжна ВИКТОРИЯ,288.3


In [ ]:
# 

txt0 = ('Сводная таблица', pt1)

print(txt0)

In [27]:
# Перечень всех судов

df1MTF['ship_name'].unique()

array(['Николай КАРАМЗИН', 'Михаил Булгаков', 'Андрей РУБЛЕВ',
       'Василий СУРИКОВ', 'И.А. КРЫЛОВ', 'Княжна ВИКТОРИЯ',
       'СЕРГЕЙ ОБРАЗЦОВ', 'Илья РЕПИН', 'Александр Грин', 'Сергей ЕСЕНИН',
       'РОССИЯ', 'А.С.Пушкин', 'Княжна  АНАСТАСИЯ'], dtype=object)

In [38]:
# тх РОССИЯ:
df_ro = dfy[dfy['ship_name'] == 'РОССИЯ']

ro = df_ro['ST2'].sum().round(1)

ro

328.7

In [29]:
# тх Николай КАРАМЗИН:
df_nk = dfy[dfy['ship_name'] == 'Николай КАРАМЗИН']

nk = df_nk['ST2'].sum().round(1)

nk


209.9

In [30]:
# тх И.А. КРЫЛОВ:
df_ik = dfy[dfy['ship_name'] == 'И.А. КРЫЛОВ']

ik = df_ik['ST2'].sum().round(1)

ik

1350.3

In [31]:
# тх Михаил Булгаков:
df_mb = dfy[dfy['ship_name'] == 'Михаил Булгаков']

mb = df_mb['ST2'].sum().round(1)

mb

255.4

In [32]:
# тх Княжна ВИКТОРИЯ:
df_kv = dfy[dfy['ship_name'] == 'Княжна ВИКТОРИЯ']

kv = df_kv['ST2'].sum().round(1)

kv

288.3

In [33]:
# тх Андрей РУБЛЕВ:
df_ar = dfy[dfy['ship_name'] == 'Андрей РУБЛЕВ']

ar = df_ar['ST2'].sum().round(1)

ar

272.5

In [34]:
# тх СЕРГЕЙ ОБРАЗЦОВ:
df_so = dfy[dfy['ship_name'] == 'СЕРГЕЙ ОБРАЗЦОВ']

so = df_so['ST2'].sum().round(1)

so

230.4

In [35]:
# тх Сергей ЕСЕНИН:
df_se = dfy[dfy['ship_name'] == 'Сергей ЕСЕНИН']

se = df_se['ST2'].sum().round(1)

se


203.8

In [36]:
# тх Александр Грин:
df_ag = dfy[dfy['ship_name'] == 'Александр Грин']

ag = df_ag['ST2'].sum().round(1)

ag

99.8

In [37]:
# тх Василий СУРИКОВ:
df_vs = dfy[dfy['ship_name'] == 'Василий СУРИКОВ']

vs = df_vs['ST2'].sum().round(1)

vs

394.4

In [40]:
# тх Илья РЕПИН:
df_ir = dfy[dfy['ship_name'] == 'Илья РЕПИН']

ir = df_ir['ST2'].sum().round(1)

ir

89.8

In [41]:
# По всем теплоходам

total = (nk + ik + mb + kv + ar + so + se + ag + vs + ir+ro).round(1)

total

3723.3

In [45]:
total2 = dfy['ST2'].sum().round(1)

total2

3723.3

In [40]:
txt_today = f"Отчет создан: {today2}"

txt_today

'Отчет создан: 2021-11-14'

In [47]:
text1 = f"Добрый день.\n\
Передаю для информации.\n\
Мостурфлот. Booking.\n\
Навигиция 2022.\n\
Продажи туров по теплоходам за вчерашний день ({yestoday2}) составила: {total} тыс.руб.,\n\
    в том числе:\n\
- Николай КАРАМЗИН:\t {nk} тыс.руб.,\n\
- И.А. КРЫЛОВ:\t\t {ik} тыс.руб.,\n\
- Михаил Булгаков:\t {mb} тыс.руб.,\n\
- Княжна ВИКТОРИЯ:\t {kv} тыс.руб.,\n\
- РОССИЯ:\t\t {ro} тыс.руб.,\n\
- Андрей РУБЛЕВ:\t {ar} тыс.руб.,\n\
- СЕРГЕЙ ОБРАЗЦОВ:\t {so} тыс.руб.,\n\
- Сергей ЕСЕНИН:\t {se} тыс.руб.,\n\
- Александр Грин:\t {ag} тыс.руб.,\n\
- Василий СУРИКОВ:\t {vs} тыс.руб.,\n\
- Илья РЕПИН:\t\t {ir} тыс.руб.,\n\
\n\
С уважением, Смоленышев Олег В.\n\
{today3}."

print(text1)

Добрый день.
Передаю для информации.
Мостурфлот. Booking.
Навигиция 2022.
Продажи туров по теплоходам за вчерашний день (2022-01-10) составила: 3723.3 тыс.руб.,
    в том числе:
- Николай КАРАМЗИН:	 209.9 тыс.руб.,
- И.А. КРЫЛОВ:		 1350.3 тыс.руб.,
- Михаил Булгаков:	 255.4 тыс.руб.,
- Княжна ВИКТОРИЯ:	 288.3 тыс.руб.,
- РОССИЯ:		 328.7 тыс.руб.,
- Андрей РУБЛЕВ:	 272.5 тыс.руб.,
- СЕРГЕЙ ОБРАЗЦОВ:	 230.4 тыс.руб.,
- Сергей ЕСЕНИН:	 203.8 тыс.руб.,
- Александр Грин:	 99.8 тыс.руб.,
- Василий СУРИКОВ:	 394.4 тыс.руб.,
- Илья РЕПИН:		 89.8 тыс.руб.,

С уважением, Смоленышев Олег В.
2022-01-11 19:37.


In [46]:
text2 = f"Добрый день.\n\
Передаю для информации.\n\
Мостурфлот. Booking.\n\
Навигиция 2022.\n\
Продажи туров по теплоходам за вчерашний день ({yestoday2}) составили: {total2} тыс.руб.,\n\
    в том числе:\n\
{pt1}\n\
\n\
С уважением, Смоленышев Олег В.\n\
{today3}."

print(text2)

Добрый день.
Передаю для информации.
Мостурфлот. Booking.
Навигиция 2022.
Продажи туров по теплоходам за вчерашний день (2022-01-10) составили: 3723.3 тыс.руб.,
    в том числе:
                     ST2
Year                2022
Month                  1
Day                   10
ship_name               
Александр Грин      99.8
Андрей РУБЛЕВ      272.5
Василий СУРИКОВ    394.4
И.А. КРЫЛОВ       1350.3
Илья РЕПИН          89.8
Княжна ВИКТОРИЯ    288.3
Михаил Булгаков    255.4
Николай КАРАМЗИН   209.9
РОССИЯ             328.7
СЕРГЕЙ ОБРАЗЦОВ    230.4
Сергей ЕСЕНИН      203.8

С уважением, Смоленышев Олег В.
2022-01-11 19:37.


In [44]:
# Отправки сообщения в Телеграм канал Smolenishev_reports

api_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

<Response [400]>

In [45]:
# создал канал MTF_reports "id":-xxxx
# отправка в канал сообщения

requests.get('https://api.telegram.org/bot{}/sendMessage'.format(api_token), params=dict(chat_id=-xxxxx, text=text1))

<Response [200]>